In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


#not in scope of this course
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from torch.utils.data import DataLoader

In [ ]:
train_dataset = CIFAR10("/", download = True, train = True, transform = T.ToTensor())
test_dataset = CIFAR10("/", download = True, train = False, transform = T.ToTensor())

100%|██████████| 170498071/170498071 [00:01<00:00, 87697272.20it/s] 


Extracting /cifar-10-python.tar.gz to /
Files already downloaded and verified


In [ ]:
bs = 32

train_loader = DataLoader(train_dataset, batch_size = bs)
test_loader = DataLoader(test_dataset, batch_size = bs)

#Tasks
#1) Create at least 4 layers NN to classify the dataset
#2) Train the neural network on train dataset
#3) Show the training loss
#4) Calculate the accuracy on test set
#5) Show a few incorrectly classified samples
#6) Calculate class-wise accuracy (implement the function yourself)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, hidden_size)
        self.layer4 = nn.Linear(hidden_size, hidden_size)
        self.layer5 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.relu(self.layer4(x))
        x = self.layer5(x)
        return x

In [ ]:
def calculate_accuracy(loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.reshape(-1, input_size)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total) * 100

In [ ]:
input_size = 3 * 32 * 32 # CIFAR-10 images are 32x32x3 (3 color channels)
hidden_size = 100
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

model = NeuralNetwork(input_size, hidden_size, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    train_accuracy = calculate_accuracy(train_loader)
    test_accuracy = calculate_accuracy(test_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')

print('Training finished')

Epoch [1/10], Loss: 2.0337, Training Accuracy: 40.24%, Test Accuracy: 40.01%
Epoch [2/10], Loss: 1.9801, Training Accuracy: 43.21%, Test Accuracy: 42.70%
Epoch [3/10], Loss: 1.9406, Training Accuracy: 44.51%, Test Accuracy: 43.84%
Epoch [4/10], Loss: 1.7289, Training Accuracy: 46.27%, Test Accuracy: 45.53%
Epoch [5/10], Loss: 1.6976, Training Accuracy: 47.07%, Test Accuracy: 45.95%
Epoch [6/10], Loss: 1.6623, Training Accuracy: 48.39%, Test Accuracy: 47.27%
Epoch [7/10], Loss: 1.6235, Training Accuracy: 48.75%, Test Accuracy: 47.14%
Epoch [8/10], Loss: 1.6789, Training Accuracy: 49.16%, Test Accuracy: 47.05%
Epoch [9/10], Loss: 1.5978, Training Accuracy: 50.17%, Test Accuracy: 46.89%
Epoch [10/10], Loss: 1.5871, Training Accuracy: 51.04%, Test Accuracy: 47.82%
Training finished


In [ ]:
def calculate_classwise_accuracy(model, data_loader, num_classes):
    class_correct = [0] * num_classes
    class_total = [0] * num_classes
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.reshape(-1, input_size)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    class_accuracy = []
    for i in range(num_classes):
        if class_total[i] == 0:
            class_accuracy.append(0.0)
        else:
            class_accuracy.append(100 * class_correct[i] / class_total[i])
    return class_accuracy

In [ ]:
num_classes = 10
classwise_accuracy = calculate_classwise_accuracy(model, test_loader, num_classes)
for i in range(num_classes):
    print(f'Class {i}: {classwise_accuracy[i]:.2f}%')

Class 0: 51.30%
Class 1: 63.90%
Class 2: 26.30%
Class 3: 43.40%
Class 4: 41.90%
Class 5: 24.20%
Class 6: 55.30%
Class 7: 52.70%
Class 8: 66.70%
Class 9: 52.50%
